In [2]:
import numpy as np
import cvxpy as cp
from general_method.basis_generator import rand_moment, generate_basis, sel_seq, rand_rho, rand_projs, proj_mul
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random

In [1]:
num_obs = 3
len_seq = 3
out_max = 1
seq_method = "sel_sequences"
remove_last_out = True

In [3]:
#behavior d=3

dim = 3

X = rand_moment(dim, num_obs, len_seq, out_max, seq_method, [len_seq], remove_last_out=remove_last_out)
B_3d = np.real(np.diag(X)[1:])

B_3d.shape

(27,)

In [25]:
#Convex Hull approximation d=2

dim = 2
B_2d = []

for i in range(1000000):
    X = rand_moment(dim, num_obs, len_seq, out_max, seq_method, [len_seq], remove_last_out=remove_last_out)
    B_2d.append(np.real(np.diag(X)[1:]))

In [26]:
#Create the LP instance...
# Adel's method

gamma = cp.Variable(B_3d.shape)
Bnd_2d = cp.Variable(1)

constraints = [B_3d @ gamma.T - Bnd_2d <= 1]
for b2d in B_2d:
    constraints += [b2d @ gamma.T - Bnd_2d <= 0]

prob = cp.Problem(cp.Maximize(B_3d @ gamma.T - Bnd_2d),
                  constraints)

print("Solving...")
prob.solve(solver=cp.MOSEK, verbose=True)

Solving...


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 1000001         
  Cones                  : 0               
  Scalar variables       : 28              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 1000001         
  Cones                  : 0               
  Scalar variables       : 28              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 22
Optimizer  - Cones                  : 0
Optimizer  - Scalar var

0.9999999997929194

In [27]:
B_3d @ gamma.value.T - Bnd_2d.value

array([1.])

In [ ]:
#Create the LP instance...
#WNR method

alpha = cp.Variable((len(B_2d),1))
eta = cp.Variable(1)

constraints = [eta*B_3d +(1-eta)/2 == cp.sum([alpha[i]*B_2d[i] for i in range(len(B_2d))])]
for i in range(len(B_2d)):
    constraints += [alpha[i] >= 0]
constraints += [cp.sum(alpha) == 1]
constraints += [eta <= 10]
constraints += [eta >= 0]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)

print("Solving...")
prob.solve(solver=cp.MOSEK, verbose=True)

In [ ]:
#Create the LP instance...
# WNR method

alpha = cp.Variable((len(B_2d),1))
beta = cp.Variable((len(B_2d),1))
eta = cp.Variable(1)

constraints = [B_3d  == cp.sum([(alpha[i]-beta[i])*B_2d[i] for i in range(len(B_2d))])]
for i in range(len(B_2d)):
    constraints += [alpha[i] >= 0]
    constraints += [beta[i] >= 0]

constraints += [cp.sum(alpha) == 1]
constraints += [cp.sum(beta) == 1 - eta]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)

print("Solving...")
prob.solve(solver=cp.MOSEK, verbose=True)